### Config Structure

- CURRENCYAPI_AUTH_TOKEN = ''
- TWILIO_ACCOUNT_SID = ''
- TWILIO_AUTH_TOKEN = ''
- PHONE_NUMBER = ''

In [1]:
import os
from twilio.rest import Client
from _config import *
import time

from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json

import pandas as pd
import requests
from bs4  import BeautifulSoup
from tqdm import tqdm

from datetime import datetime

In [2]:
currenciesList = ['UYU', 'BTC', 'ETH']

In [3]:
def generate_url():
    api_key = CURRENCYAPI_AUTH_TOKEN
    base_currency = 'USD'
    currenciesStr = ','.join(currenciesList)
    
    url_currencies = "https://api.currencyapi.com/v3/latest?apikey={0}&base_currency={1}&currencies={2}".format(api_key,base_currency,currenciesStr)

    return url_currencies

def get_Response_Currencies():
    url_currencies = generate_url()
    
    response = requests.get(url_currencies).json()    
    return response

def convertResult(res):
    datos = []

    for elem in currenciesList:
        code = res['data'][elem]['code']
        value = res['data'][elem]['value'] if(code == "UYU") else 1/(res['data'][elem]['value'])
        
        datos.append((value, code))

    return datos

In [4]:
def enviar_SMS(mensaje):
    time.sleep(2)
    account_sid = TWILIO_ACCOUNT_SID 
    auth_token = TWILIO_AUTH_TOKEN

    client = Client(account_sid, auth_token)

    message = client.messages \
                    .create(
                         body = mensaje,
                         from_ = PHONE_NUMBER,
                         to = '+598 95 351 882'
                     )

    print('Mensaje Enviado ' + message.sid)

def enviarWhatsapp(mensaje):
    account_sid = TWILIO_ACCOUNT_SID
    auth_token = TWILIO_AUTH_TOKEN
    client = Client(account_sid, auth_token)

    message = client.messages.create(
      from_ ='whatsapp:+14155238886',
      body = mensaje,
      to = 'whatsapp:+59895351882'
    )

    print(message.sid)

In [5]:
res = get_Response_Currencies()
dataMonedas = convertResult(res)

cols = ["Value", "Code"]
dfDatos = pd.DataFrame(dataMonedas, columns=cols)
dfDatos

,Value,Code
0,43.922856,UYU
1,93682.080492,BTC
2,3352.454617,ETH


In [6]:
strMensaje = '\nHola! \n\n\n Las cotizaciones del día de hoy en Uruguay son: \n\n\n {0}'.format(dfDatos)

print(strMensaje)


Hola! 


 Las cotizaciones del día de hoy en Uruguay son: 


           Value Code
0     43.922856  UYU
1  93682.080492  BTC
2   3352.454617  ETH


In [8]:
enviarWhatsapp(strMensaje)

SMc843936c690e6149d5681e7a83cdc156
